<a href="https://colab.research.google.com/github/AbishekAmbrose/Simple-Reinforcement-Learning-with-Tensorflow/blob/master/PG_stockEnv_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import datetime
datetime.datetime.today()
Training_start_date = "01/01/2021"
Training_end_date = "07/01/2022"
#p = datetime.datetime(2021, 1, 5, 23, 24, 55, 173504).weekday()
#print(p)
#p = datetime.datetime.today().weekday()
s_ticker = "QCOM"
EPISODES = 5000
stock_list_array_nasdaq100_INDEX = 80;
UseDefalt_ticker = True
Train_up_oad = True
Load_model = True
Test_model = False
printfirst_return = False
model_file_name = './pgstock_model.h5'
#model_file_name = 'Cartpole_model.h5'
if Train_up_oad == True:
  from google.colab import drive
  drive.mount('/content/drive')
  !cp  '/content/drive/My Drive/StockResultSets/pgstock_model.h5' pgstock_model.h5


!pip install yahoo_fin --upgrade --quiet
from yahoo_fin.stock_info import get_analysts_info

from yahoo_fin.stock_info import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat '/content/drive/My Drive/StockResultSets/pgstock_model.h5': No such file or directory


In [76]:
def dividend_percentage (divi):
  s = divi.find("(")
  e = divi.find("%")
  val = divi[s+1:e]
  #print(val)
  try:
    return float(val)
  except:
    #print(divi + 'Exception')
    return 0.0

#dividend = dividend_percentage(divi)
#print(dividend)
#type(dividend)

def maxtoquote(yearMa,QuotePrice):
  s = yearMa.find("-")
  try:
    maxVal = float(yearMa[s+2:])
  except:
    yearMa = yearMa.replace(',', '')
    s = yearMa.find("-")
    maxVal = float(yearMa[s+2:])
    print('maxtoquote',yearMa)
    print(QuotePrice)
    

  return round(QuotePrice/maxVal,2)

def mintoquote(yearMa,QuotePrice):
  s = yearMa.find(" ")
  #print(yearMa)

 

  #print('qw',s)
  try:
    minVal = float(yearMa[:s])
  except:
    yearMa = yearMa.replace(',', '')
    s = yearMa.find(" ")
    minVal = float(yearMa[:s])
    print('mintoquote',yearMa)
    #print(s,s1)
  #print(minVal)
  #print(round(QuotePrice/minVal,2))
  return round(QuotePrice/minVal,2)

#maxVal = maxtoquote(yearMa,QuotePrice)
#print(maxVal)
#type(maxVal)

def targettoquote(target,QuotePrice):
  return round(QuotePrice/target,2)

#tarvak = targettoquote(target,QuotePrice)
#print(tarvak)
#type(tarvak)

def marketCapital(marketcap):
    return_value = 0
    #print(marketcap)
    try:
      s = marketcap.find("B")
    except:
      print('marketCapital')
      return 0.0
    if s != -1:
      marketcap_val = marketcap[:s]
      return_value = float(marketcap_val)
    else:
      s = marketcap.find("M")
      if s != -1:
        marketcap_val = marketcap[:s]
        return_value = float(marketcap_val)
        return_value = round(return_value/1000,2)
      else:
        s = marketcap.find("T")
        if s != -1:
          marketcap_val = marketcap[:s]
          return_value = float(marketcap_val)
          return_value = round(return_value*1000,2)
        else:
          print('Except')
    return return_value
  
#marketcapfloat = marketCapital(marketcap)
#marketcapfloat

def volumeavergae(vol,QuotePrice):
  return round((QuotePrice*vol)/1000,2)



In [77]:
import datetime
def p_year(index):
  #2019-01-02 00:00:00
  s = index.find("-")
  #print(index[:s])
  return int(index[:s])

def p_mon(index):
  #2019-01-02 00:00:00
  s = index.find("-")
  #print(index[s+1:s+3])
  return int(index[s+1:s+3])

def p_day(index):
  #2019-01-02 00:00:00
  s = index.find("-")
  #print(index[s+4:s+6])
  return int(index[s+4:s+6])

def get_approxPEratio(TodayPE,TodatQuote,SampleQuote): #calsute the PE using the currest EPS and sample share price
  eps = TodatQuote/TodayPE
  #print(TodayPE,TodatQuote,eps,SampleQuote)
  sample_pe = SampleQuote/eps
  #print(sample_pe)
  #apprximate the sample PE somehow
  sample_pe = (TodayPE + sample_pe)/2
  #print(sample_pe)
  return sample_pe

def get_approxMarCap(MC,TodayQuote,SampleQuote): #Calculate Market cap
  #print('get_approxMarCap')
  #print(MC,TodayQuote,SampleQuote)
  num_share = MC / TodayQuote
  #print(num_share)
  sample_Mc = num_share * SampleQuote
  #print(sample_Mc)
  return sample_Mc

In [78]:
db = get_quote_table(s_ticker)
samples_df= get_data(s_ticker, start_date=Training_start_date, end_date=Training_end_date)
samples_df_total_rows = len(samples_df.index)
print(samples_df_total_rows)
samples_df_current_row = 0

def change_database(new_s_ticker):
  global db
  global samples_df
  print(new_s_ticker)
  #print(samples_df)
  db = get_quote_table(new_s_ticker)
  samples_df= get_data(new_s_ticker, start_date=Training_start_date, end_date=Training_end_date)
  samples_df_total_rows = len(samples_df.index)

stock_list_array_nasdaq100 = tickers_sp500()
stock_list_array_nasdaq100_maxIndex = len(stock_list_array_nasdaq100)


def update_database():
  global printfirst_return
  printfirst_return = False
  global stock_list_array_nasdaq100
  global stock_list_array_nasdaq100_INDEX
  change_database(stock_list_array_nasdaq100[stock_list_array_nasdaq100_INDEX])
  stock_list_array_nasdaq100_INDEX = stock_list_array_nasdaq100_INDEX + 1
  if (stock_list_array_nasdaq100_INDEX >= stock_list_array_nasdaq100_maxIndex):
    stock_list_array_nasdaq100_INDEX = 0


[D:urllib3.connectionpool] Starting new HTTPS connection (%d): %s:%s
[D:urllib3.connectionpool] %s://%s:%s "%s %s %s" %s %s
[D:urllib3.connectionpool] Starting new HTTPS connection (%d): %s:%s
[D:urllib3.connectionpool] %s://%s:%s "%s %s %s" %s %s
[D:urllib3.connectionpool] Starting new HTTPS connection (%d): %s:%s
[D:urllib3.connectionpool] %s://%s:%s "%s %s %s" %s %s


257


In [79]:


def env_get_state(val):
  newOutPutList = np.zeros(9,)
  row = samples_df.iloc[val]
  index = samples_df.iloc[val].name
  newOutPutList[0] = get_approxPEratio(db['PE Ratio (TTM)'],round(db['Quote Price'],2),row['close'])
  newOutPutList[1] = get_approxMarCap(round(marketCapital(db['Market Cap']),2),round(db['Quote Price'],2),row['close'])
  newOutPutList[2] = round(volumeavergae(row['volume'],row['close']),2)
  newOutPutList[3] = round(maxtoquote(db['52 Week Range'],row['close']),2)
  newOutPutList[4] = round(targettoquote(db['1y Target Est'],row['close']),2)
  newOutPutList[5] = dividend_percentage(db['Forward Dividend & Yield'])
  newOutPutList[6] = round(row['close'],2)
  newOutPutList[7] = round(mintoquote(db['52 Week Range'],row['close']),2)
  newOutPutList[8] = (datetime.datetime(p_year(str(index)), p_mon(str(index)), p_day(str(index)), 1, 1, 1, 1).weekday())
  for i in range(9):
    if(np.isnan(newOutPutList[i])):
      #print(i)
      newOutPutList[i]  = 0;

  return newOutPutList

def env_stock_reset():
  global samples_df_current_row
  samples_df_current_row = 0
  newOutPutList = env_get_state(samples_df_current_row)
  samples_df_current_row = samples_df_current_row + 1
  return newOutPutList

def env_stock_step(action):# buy 1  ignore 0
  reward = 0
  done = 0
  stock_price = []
  global samples_df_current_row
  global samples_df_total_rows
  if(samples_df_current_row == 0):
    print('Call rest first')
    return 0,0,0
  info = 0
  i = samples_df_current_row
  #print(samples_df_current_row)
  row = samples_df.iloc[samples_df_current_row - 1]
  state_price = row['close']
  P4_visit = 0
  P7_visit = 0
  P10_visit = 0
  P4_visit1 = 0
  P7_visit1 = 0
  P10_visit1 = 0
  #print(row)
  for i in range(10):
    if ((i+samples_df_current_row) == samples_df_total_rows):
      done = 1
      break
    row = samples_df.iloc[i+samples_df_current_row]
    stock_price.append(row['close'])
    percentage = stock_price[i] / state_price

    #print(percentage,stock_price[i],state_price)
    info = round(percentage,2)
    if(percentage >= 1):       
        percentage = percentage - 1
        percentage = percentage * 100
        if ((percentage > 4) and (reward < 0)):#if it went low first
          reward = 0
        
        #print(percentage)
        if ((percentage > 4) and (P4_visit == 0)and (P7_visit == 0)and (P10_visit == 0)):
          P4_visit = 1
          #print('A:',info,i)
          if i < 3:
            reward = reward + 10
          elif i < 6:
            reward = reward + 5
          elif i > 5:
            reward = reward + 2
        if ((percentage > 7) and (P7_visit == 0)and (P10_visit == 0)):
          P7_visit = 1
          #print('B:',info,i)
          if i < 3:
            reward = reward + 20
          elif i < 6:
            reward = reward + 15
          elif i > 5:
            reward = reward + 10
        if ((percentage > 10) and (P10_visit == 0)):
          P10_visit = 1
          #print('C:',info,i)
          if i < 3:
            reward = reward + 30
          elif i < 6:
            reward = reward + 25
          elif i > 5:
            reward = reward + 20
        #print(P4_visit,P7_visit,P10_visit,reward,stock_price[i],state_price)
    else:
        info = percentage
        percentage = percentage * 100
        percentage = 100 - percentage
        #print(percentage)
        if ((percentage > 4) and (P4_visit1 == 0)and (P7_visit1 == 0)and (P10_visit1 == 0)and (reward > 0)):
          P4_visit1 = 1
          #print('-A:',info,i)
          if i < 3:
            reward = reward - 10
          elif i < 6:
            reward = reward - 5
          elif i > 5:
            reward = reward - 2
        if ((percentage > 7) and (P7_visit1 == 0)and (P10_visit1 == 0)and (reward > 0)):
          P7_visit1 = 1
          #print('-B:',info,i)
          if i < 3:
            reward = reward - 20
          elif i < 6:
            reward = reward - 15
          elif i > 5:
            reward = reward - 10
        if ((percentage > 10) and (P10_visit1 == 0)and (reward > 0)):
          P10_visit1 = 1
          #print('-C:',info,i)
          if i < 3:
            reward = reward - 30
          elif i < 6:
            reward = reward - 25
          elif i > 5:
            reward = reward - 20
    #print('F:',reward)
        #print(P4_visit,P7_visit,P10_visit,reward,stock_price[i],state_price)
  #if(reward == 0 and action == 1):
  #  reward = -30
  if(action == 0):
    if(reward < 0):
      reward = 0
    else:
      reward = -reward -30
  elif (reward == 0):
    reward = -30
  elif (reward < 0):
    reward = reward - 30
  
  reward = reward/10
  #print(reward)
  next_state = env_get_state(samples_df_current_row)
  samples_df_current_row = samples_df_current_row + 1      
  #print(next_state)
  #print(action,reward,info)
  return next_state,reward,done,info


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
#import torch.sigmoid as FS
from torch.distributions import Bernoulli
from torch.autograd import Variable
from itertools import count
import matplotlib.pyplot as plt
import numpy as np
import gym
import pdb
#nn.functional.sigmoid


class PolicyNet(nn.Module):
    def __init__(self):
        super(PolicyNet, self).__init__()
        self.fc1 = nn.Linear(9, 24)
        #self.fc1 = nn.Linear(4, 24)
        self.fc2 = nn.Linear(24, 36)
        self.fc3 = nn.Linear(36, 1)  # Prob of Left

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x


'''
class PolicyNet(nn.Module):
    def __init__(self):
        super(PolicyNet, self).__init__()

        #self.fc1 = nn.Linear(9, 24)
        self.fc1 = nn.Linear(4, 24)
        self.fc2 = nn.Linear(24, 36)
        self.fc3 = nn.Linear(36, 72)
        self.fc4 = nn.Linear(72, 72)
        self.fc5 = nn.Linear(72, 36)
        self.fc6 = nn.Linear(36, 1)  # Prob of Left

    def forward(self, x):
        #print('forward',x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = torch.sigmoid(self.fc6(x))
        return x
'''

def main():

    # Plot duration curve: 
    # From http://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
    episode_durations = []
    cartpole = False
    def plot_durations():
        plt.figure(2)
        plt.clf()
        durations_t = torch.FloatTensor(episode_durations)
        plt.title('Training...')
        plt.xlabel('Episode')
        plt.ylabel('Duration')
        plt.plot(durations_t.numpy())
        # Take 100 episode averages and plot them too
        if len(durations_t) >= 100:
            means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
            means = torch.cat((torch.zeros(99), means))
            plt.plot(means.numpy())
        plt.savefig("./cartpole_reinforce.png")
        #plt.pause(0.001)  # pause a bit so that plots are updated

    # Parameters
    num_episode = EPISODES
    batch_size = 5
    learning_rate = 0.01
    gamma = 0.99

    if cartpole:
      env = gym.make('CartPole-v0')
    policy_net = PolicyNet()
    try:
      policy_net.load_state_dict(torch.load(model_file_name))
    except:
      pass
    optimizer = torch.optim.RMSprop(policy_net.parameters(), lr=learning_rate)

    # Batch History
    state_pool = []
    action_pool = []
    reward_pool = []
    steps = 0


    for e in range(num_episode):
        global UseDefalt_ticker
        global printfirst_return
        printfirst_return = False
        if UseDefalt_ticker == False:
          update_database()#change the stock symbl each time
        if cartpole:
          state = env.reset()
        else:
          state = env_stock_reset()

        print(state)
        state = torch.from_numpy(state).float()
        print(state)
        state = Variable(state)
        #env.render(mode='rgb_array')
        print(state)
        total_reward = 0
        for t in count():
            #print(state)
            probs = policy_net(state)
            m = Bernoulli(probs)
            action = m.sample()
            print(m,action)
            action = action.data.numpy().astype(int)[0]
            print(action)
            if cartpole:
              next_state, reward, done, _ = env.step(action)
            else:          
              next_state, reward, done, info = env_stock_step(action)
            #env.render(mode='rgb_array')

            # To mark boundarys between episodes
            if done:
                reward = 0

            state_pool.append(state)
            action_pool.append(float(action))
            reward_pool.append(reward)

            state = next_state
            state = torch.from_numpy(state).float()
            state = Variable(state)
            total_reward = total_reward + reward
            steps += 1

            if done:
              if cartpole:
                #episode_durations.append(total_reward)
                episode_durations.append(t+1)
              else:
                episode_durations.append(total_reward)
                plot_durations()
                break

        # Update policy
        if e > 0 and e % batch_size == 0:

            # Discount reward
            running_add = 0
            for i in reversed(range(steps)):
                if reward_pool[i] == 0:
                    running_add = 0
                else:
                    running_add = running_add * gamma + reward_pool[i]
                    reward_pool[i] = running_add

            # Normalize reward
            reward_mean = np.mean(reward_pool)
            reward_std = np.std(reward_pool)
            for i in range(steps):
                reward_pool[i] = (reward_pool[i] - reward_mean) / reward_std

            # Gradient Desent
            optimizer.zero_grad()

            for i in range(steps):
                state = state_pool[i]
                action = Variable(torch.FloatTensor([action_pool[i]]))
                reward = reward_pool[i]

                probs = policy_net(state)
                m = Bernoulli(probs)
                loss = -m.log_prob(action) * reward  # Negtive score function x reward
                loss.backward()

            optimizer.step()

            state_pool = []
            action_pool = []
            reward_pool = []
            steps = 0
            torch.save(policy_net.state_dict(), model_file_name)

if __name__ == '__main__':
    main()

Streaming output truncated to the last 5000 lines.
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) tensor([0.])
0
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) tensor([0.])
0
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) tensor([0.])
0
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) tensor([0.])
0
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) tensor([0.])
0
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) tensor([0.])
0
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) tensor([0.])
0
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) tensor([0.])
0
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) tensor([0.])
0
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) tensor([0.])
0
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) tensor([0.])
0
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) tensor([0.])
0
Bernoulli(probs: tensor([0.], grad_fn=<SigmoidBackward0>)) te